# Visualization and MODIS data


<div class="alert-info">

### Overview
    
* **teaching:** 15 minutes
* **exercises:** 0
* **questions:**
    * How can I find, anaylize, and visualize satellite imagery for an area of interest using Python?
    * Distinguish between tiled versus analytic raster products from the MODIS sensor
    
</div>


This notebook will focus on accessing public datasets on AWS for a target area affected by Cyclone Kenneth (2019-04-25). Read more about this event and its impact at the [Humanitarian Open Street Map website](https://tasks.hotosm.org/project/5977). We will use a bounding box we will work with covers the island of Nagazidja, including the captial [city of Moroni](https://en.wikipedia.org/wiki/Moroni,_Comoros) - Union of the Comoros, a sovereign archipelago nation in the Indian Ocean. 

We will examine raster images from the [MODIS](https://modis.gsfc.nasa.gov/data/) instrument. "MODIS" stands for "MODerate Resolution SpectroRadiometer". Moderate resolution refers to the fact that MODIS data has at best a 250 meter pixel posting, but single images cover hundreds of kilometers and with two sensors currently in orbit, we currently get [daily views of the entire globe](https://worldview.earthdata.nasa.gov/)!

### Table of contents

1. [**Leaflet slippy maps**](#Leaflet-slippy-maps)
1. [**NASA GIBS**](#NASA-GIBS)
1. [**Jupyter Widgets**](#Jupyter-Widgets)
1. [**Analytical Data**](#Analytical-Data)

In [ ]:
# Import python libraries we'll work with 
import ipyleaflet
from ipyleaflet import Map, Rectangle, basemaps, basemap_to_tiles, TileLayer, SplitMapControl, Polygon

import ipywidgets
import datetime
import re

In [ ]:
bbox = [43.16, -11.32, 43.54, -11.96]
west, north, east, south = bbox
bbox_ctr = [0.5*(north+south), 0.5*(west+east)]

## Leaflet slippy maps

In [ ]:
m = Map(center=bbox_ctr, zoom=6)
rectangle = Rectangle(bounds=((south, west), (north, east))) #SW and NE corners of the rectangle (lat, lon)
m.add_layer(rectangle)
m

## NASA GIBS

NASA's [Global Imagery Browse Services (GIBS)](https://earthdata.nasa.gov/eosdis/science-system-description/eosdis-components/gibs) is a great Web Map Tile Service (WMTS) to visualize NASA data as pre-rendered tiled raster images. The [NASA Worldview web application](https://worldview.earthdata.nasa.gov/) is a way to explore all GIBS datasets. We can also use `ipyleaflet` to explore GIBS datasets, like MODIS truecolor images, within a Jupyter Notebook. Use the slider in the image below to view the image.

In [ ]:
m = Map(center=bbox_ctr, zoom=6)# MODIS great for large areas (onl)

right_layer = basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, "2019-04-25")
left_layer = TileLayer()
control = SplitMapControl(left_layer=left_layer, right_layer=right_layer)
m.add_control(control)

m.add_layer(rectangle)

m

## Jupyter Widgets

With Jupyter Widgets we can quickly built simple but powerful visualization tools. This next code cell lets you easily select a MODIS image from any given date

<div class="alert-warning">

#### Note
    
* Date selection widget doesn't currently work on Safari browser ([GitHub issue])(https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html#Date-picker), and may require selecting a date twice to display a new image
* MODIS true color has a [max zoom level of 9](https://wiki.earthdata.nasa.gov/display/GIBS/Map+Library+Usage#expand-GDALBasics) corresponding to 250m resolution. As you zoom in closer, the tiled imagery dissapears
</div>

In [ ]:
m = Map(center=bbox_ctr, zoom=6)

#right_layer = basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR) 
right_layer = basemap_to_tiles(basemaps.NASAGIBS.ModisAquaTrueColorCR, "2019-04-25") #switch to aqua
currentDate = re.search('\d{4}-\d{2}-\d{2}', right_layer.url).group()

left_layer = TileLayer()
control = SplitMapControl(left_layer=left_layer, right_layer=right_layer)
m.add_control(control)

m.add_layer(rectangle)

# Currently datepicker is a bit buggy https://github.com/jupyter-widgets/ipywidgets/pull/2433
date_picker = ipywidgets.DatePicker(description='Select Imagery Date: ', style=dict(description_width='initial'))
label = ipywidgets.Label(value=currentDate)

def validate(change):
    # javascript months indexed at 0
    change['new']['value']['month'] += 1
    oldDate = datetime.date(*change['old']['value'].values())
    newDate = datetime.date(*change['new']['value'].values())
    today = datetime.date.today()
    launch = datetime.date(2000,1,1)  
    if newDate < today and newDate > launch:
        datestr = newDate.strftime('%Y-%m-%d')
        label.set_trait('value', f'{datestr}')

def update_map(change):
    currentDate = re.search('\d{4}-\d{2}-\d{2}', right_layer.url).group()
    newDate = change['new']
    newUrl = right_layer.url.replace(currentDate, newDate)
    right_layer.set_trait('url', newUrl)

date_picker.observe(validate)
label.observe(update_map)

display(date_picker, label, m)

## Analytical MODIS Data

There is a limited amount of MODIS data currently [stored on AWS](https://registry.opendata.aws/modis/). Alternatively, with a [NASA EarthData login](https://urs.earthdata.nasa.gov/), you can access the entire archive with the [EarthData Search](https://search.earthdata.nasa.gov/) web interface, or programatically with [NASA CMR]().

**Excercise:** Find and open a single band of MODIS data covering our area of interest. Use GDAL and rasterio. 